<a href="https://colab.research.google.com/github/sshharma/assignments/blob/master/ir_assignment3_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-terrier

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.4/347.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py)

In [2]:
# Initialing ===================================================================
import pyterrier as pt
if not pt.started():
    pt.init()

terrier-assemblies 5.9 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8



In [4]:
# Loading TREC dataset =========================================================
dataset = pt.datasets.get_dataset("irds:beir/trec-covid")
# dataset.info
dataset.get_topics().head()

[INFO] [starting] opening zip file
[INFO] If you have a local copy of https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/trec-covid.zip, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/ce62140cb23feb9becf6270d0d1fe6d1
[INFO] [starting] https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/trec-covid.zip
[INFO] [finished] https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/trec-covid.zip: [00:07] [73.9MB] [10.4MB/s]
[INFO] [finished] opening zip file [8.33s]


There are multiple query fields available: ('text', 'query', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.


,qid,text,query,narrative
0,1,what is the origin of COVID-19,coronavirus origin,seeking range of information about the SARS-Co...
1,2,how does the coronavirus respond to changes in...,coronavirus response to weather changes,seeking range of information about the SARS-Co...
2,3,will SARS-CoV2 infected people develop immunit...,coronavirus immunity,seeking studies of immunity developed due to i...
3,4,what causes death from Covid-19?,how do people die from the coronavirus,Studies looking at mechanisms of death from Co...
4,5,what drugs have been active against SARS-CoV o...,animal models of covid 19,Papers that describe the results of testing d...


In [5]:
dataset.get_qrels().head()

[INFO] [starting] opening zip file
[INFO] [finished] opening zip file s]


,qid,docno,label,iteration
0,1,005b2j4b,2,0
1,1,00fmeepz,1,0
2,1,g7dhmyyo,2,0
3,1,0194oljo,1,0
4,1,021q9884,1,0


In [6]:
# Indexing using pyterrier =====================================================
import os
pt_index_path = './indices/cord19'

if not os.path.exists(pt_index_path + "/data.properties"):
    indexer = pt.index.IterDictIndexer(pt_index_path, blocks=True)
    index_ref = indexer.index(dataset.get_corpus_iter(),
                              fields=['title', 'text'],
                              meta=('docno',))

else:
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")

index = pt.IndexFactory.of(index_ref)

[INFO] [starting] building docstore
[INFO] [starting] opening zip file
[INFO] [finished] opening zip file s]
docs_iter: 100%|████████████████████| 171332/171332 [00:10<00:00, 16342.43doc/s]
[INFO] [finished] docs_iter: [00:10] [171332doc] [16340.24doc/s]
[INFO] [finished] building docstore [10.50s]


beir/trec-covid documents:   0%|          | 0/171332 [00:00<?, ?it/s]

<ipython-input-6-18b064a9d30e>:6: DeprecationWarning: specifying meta and meta_lengths in IterDictIndexer.index() is deprecated, use constructor instead
  index_ref = indexer.index(dataset.get_corpus_iter(),


01:25:45.296 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (8is9x9sc) - further warnings are suppressed
01:27:47.795 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 17 empty documents


In [7]:
# Statics from indexes =========================================================
print(index.getCollectionStatistics())

Number of documents: 171332
Number of terms: 158398
Number of postings: 11497332
Number of fields: 2
Number of tokens: 18425387
Field names: [title, text]
Positions:   true



In [8]:
# Runnnig BM25 Retriever =======================================================
bm25 = pt.BatchRetrieve(index_ref, wmodel="BM25")
topics = dataset.get_topics('query')

res = bm25.transform(topics)
res

01:30:41.925 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based models such as BM25F, change to index.document.class in the index  properties file to FSAFieldDocumentIndex or FSADocumentIndexInMemFields to support efficient retrieval. If you don't use (e.g.) BM25F, this warning can be ignored


,qid,docid,docno,rank,score,query
0,1,113088,75773gwg,0,11.663878,coronavirus origin
1,1,122737,ne5r4d4b,1,11.663878,coronavirus origin
2,1,111746,kqqantwg,2,11.504715,coronavirus origin
3,1,143600,12dcftwt,3,11.504715,coronavirus origin
4,1,80809,8ccl9aui,4,11.446890,coronavirus origin
...,...,...,...,...,...,...
49995,50,165839,6ojmmmuj,995,9.254295,mrna vaccine coronavirus
49996,50,78177,qeo0o93l,996,9.252758,mrna vaccine coronavirus
49997,50,123344,xxdrjngv,997,9.250896,mrna vaccine coronavirus
49998,50,127011,bwli875x,998,9.249313,mrna vaccine coronavirus


In [9]:
qrels = dataset.get_qrels()

eval_metrics=['P_10', 'ndcg_cut_10', 'map']
exp_res = pt.Experiment(
    [bm25],
    topics,
    qrels,
    eval_metrics=eval_metrics,
)
exp_res

,name,P_10,ndcg_cut_10,map
0,BR(BM25),0.664,0.592498,0.213273
